In [1]:
from generator import LowShotGenerator
from classifier import Classifier
from train import get_trained_classifier_and_data

import numpy as np
import data_utils as du
import collect

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
n_clusters = 20
n_files = 12
λ = 1.
n_samples = 1
n_examples = 20
disease_name = 'Hernia'
all_diseases = list(range(15))

In [3]:
data_obj = du.get_processed_data(n_files)
le = du.get_label_encoder(data_obj)

fetching data from file #1
fetching data from file #2
fetching data from file #3
fetching data from file #4
fetching data from file #5
fetching data from file #6
fetching data from file #7
fetching data from file #8
fetching data from file #9
fetching data from file #10
fetching data from file #11
fetching data from file #12


In [4]:
disease_label_int = le.transform((disease_name,))[0]
print('{0}\'s label is {1}'.format(disease_name, disease_label_int))

unused_diseases = [disease_label_int]
diseases_to_remove = [disease_name]

diseases = [d for d in all_diseases if d not in unused_diseases]
print('All remaining diseases: {0}'.format(', '.join([str(l) for l in diseases])))

Hernia's label is 7
All remaining diseases: 0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14


In [5]:
classifier, X_train, X_test, y_train, y_test = get_trained_classifier_and_data(diseases_to_remove, n_files=n_files)
classifier.toggle_trainability()  # make the classifier non-trainable

Loaded classifier and data from files
Loaded classifier weights from a saved model
Classifier is now non-trainable!


In [6]:
quadruplets_data = collect.load_quadruplets(n_clusters=n_clusters, categories=diseases, n_files=n_files)

Loaded centroids from file


In [7]:
lsg_name = 'lsg_f.{0}_c.{1}_w.{2}'.format(n_files, n_clusters, '.'.join([str(d) for d in unused_diseases]))
lsg = LowShotGenerator(classifier.model, quadruplets_data, λ=λ, name=lsg_name)

Generator summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 6144)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               3146240   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              1050624   
Total params: 4,459,520
Trainable params: 4,459,520
Non-trainable params: 0
_________________________________________________________________
None

Whole model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 6144)              0         
____________________

In [8]:
lsg.fit()

Fitting generator
Epoch 1/10
47088/47088 [==============================] - 19s 399us/step - loss: 5.9223 - dense_3_loss: 3.1068 - model_2_loss: 2.8155 - dense_3_acc: 0.4267 - model_2_acc: 0.0750
Epoch 2/10
47088/47088 [==============================] - 18s 391us/step - loss: 3.1213 - dense_3_loss: 0.4730 - model_2_loss: 2.6484 - dense_3_acc: 0.6574 - model_2_acc: 0.0738
Epoch 3/10
47088/47088 [==============================] - 19s 395us/step - loss: 3.1185 - dense_3_loss: 0.4727 - model_2_loss: 2.6458 - dense_3_acc: 0.6574 - model_2_acc: 0.0741
Epoch 4/10
47088/47088 [==============================] - 19s 398us/step - loss: 3.1164 - dense_3_loss: 0.4727 - model_2_loss: 2.6438 - dense_3_acc: 0.6574 - model_2_acc: 0.0773
Epoch 5/10
47088/47088 [==============================] - 19s 399us/step - loss: 3.1169 - dense_3_loss: 0.4726 - model_2_loss: 2.6442 - dense_3_acc: 0.6574 - model_2_acc: 0.0756
Epoch 6/10
47088/47088 [==============================] - 19s 395us/step - loss: 3.1144 - de

In [15]:
unused_data = collect.load_quadruplets(n_clusters=n_clusters, categories=all_diseases, n_files=n_files)
quadruplets, centroids, cat_to_vectors, original_shape = unused_data

Loaded centroids from file


In [16]:
print('Generating {0} examples from {1} samples of {2}'.format(n_examples, n_samples, disease_label_int))

Generating 20 examples from 1 samples of 7


In [56]:
# get all data of all diseases
X, y = du.get_features_and_labels(data_obj)
X_train, X_test, y_train, y_test = du.get_train_test_split(X, y, test_size=0.1)

# remove the data of the disease
mask = ~(y_test[:, disease_label_int] == 1.0)
y_test_sub = y_test[mask]
X_test_sub = X_test[mask]

mask = ~(y_train[:, disease_label_int] == 1.0)
y_train_sub = y_train[mask]
X_train_sub = X_train[mask]

# keep the data of the disease
mask = (y_test[:, disease_label_int] == 1.0)
y_test_disease = y_test[mask]
X_test_disease = X_test[mask]

mask = (y_train[:, disease_label_int] == 1.0)
y_train_disease = y_train[mask]
X_train_disease = X_train[mask]

y_disease = np.concatenate((y_train_disease, y_test_disease))
X_disease = np.concatenate((X_train_disease, X_test_disease))

# split the data of the disease (should train only with n_samples)
mask = np.ones(len(y_disease), dtype=bool)
mask[list(range(n_samples))] = False
y_train_disease, y_test_disease = y_disease[~mask], y_disease[mask]
X_train_disease, X_test_disease = X_disease[~mask], X_disease[mask]

print('Training with {0} samples'.format(len(X_train_disease)))
print('Testing on {0} samples'.format(len(X_test_disease)))

Training with 1 samples
Testing on 109 samples


In [27]:
# first, try to train with only n_samples of the disease and the rest samples of allother diseases
X_train, y_train = np.concatenate((X_train_sub, X_train_disease)), np.concatenate((y_train_sub, y_train_disease))

classifier = Classifier(trainable=True, n_classes=15)
classifier.fit(X_train, y_train)

Train on 73881 samples, validate on 8209 samples
Epoch 1/100
73881/73881 [==============================] - 6s 78us/step - loss: 2.3993 - acc: 0.1395 - val_loss: 8.7422 - val_acc: 0.1722
Epoch 2/100
73881/73881 [==============================] - 5s 71us/step - loss: 2.2378 - acc: 0.1634 - val_loss: 9.5380 - val_acc: 0.1663
Epoch 3/100
73881/73881 [==============================] - 5s 70us/step - loss: 2.1791 - acc: 0.1712 - val_loss: 10.2062 - val_acc: 0.2228
Epoch 4/100
73881/73881 [==============================] - 5s 71us/step - loss: 2.1377 - acc: 0.1763 - val_loss: 10.7277 - val_acc: 0.2273
Epoch 5/100
73881/73881 [==============================] - 5s 72us/step - loss: 2.1062 - acc: 0.1799 - val_loss: 11.3148 - val_acc: 0.1844
Epoch 6/100
73881/73881 [==============================] - 5s 73us/step - loss: 2.0823 - acc: 0.1849 - val_loss: 11.7909 - val_acc: 0.1370
Epoch 7/100
73881/73881 [==============================] - 5s 71us/step - loss: 2.0591 - acc: 0.1878 - val_loss: 12.378

73881/73881 [==============================] - 5s 73us/step - loss: 1.6597 - acc: 0.2373 - val_loss: 12.8886 - val_acc: 0.2120
Epoch 60/100
73881/73881 [==============================] - 5s 73us/step - loss: 1.6553 - acc: 0.2398 - val_loss: 12.8994 - val_acc: 0.1673
Epoch 61/100
73881/73881 [==============================] - 5s 74us/step - loss: 1.6527 - acc: 0.2384 - val_loss: 12.8910 - val_acc: 0.1776
Epoch 62/100
73881/73881 [==============================] - 6s 75us/step - loss: 1.6467 - acc: 0.2379 - val_loss: 12.8798 - val_acc: 0.2061
Epoch 63/100
73881/73881 [==============================] - 6s 75us/step - loss: 1.6443 - acc: 0.2377 - val_loss: 12.9058 - val_acc: 0.2123
Epoch 64/100
73881/73881 [==============================] - 6s 76us/step - loss: 1.6422 - acc: 0.2393 - val_loss: 12.9025 - val_acc: 0.1798
Epoch 65/100
73881/73881 [==============================] - 6s 74us/step - loss: 1.6363 - acc: 0.2391 - val_loss: 12.8894 - val_acc: 0.1986
Epoch 66/100
73881/73881 [=======

In [28]:
loss, acc = classifier.evaluate(X_test_sub, y_test_sub)
print('accuracy for regular diseases is {0}'.format(acc))

loss, acc = classifier.evaluate(X_test_disease, y_test_disease)
print('accuracy for novel disease {0} without generated examples is {1}'.format(disease_label_int, acc))

9125/9125 [==============================] - 0s 39us/step
accuracy for regular diseases is 0.2127123287671233
109/109 [==============================] - 0s 72us/step
accuracy for novel disease 7 without generated examples is 0.0


In [57]:
# now, generating new examples and concat them to the n_samples samples
new_examples = [lsg.generate(ϕ, n_new=(n_examples - n_samples) // n_samples) for ϕ in X_train_disease]
X_train_disease = np.concatenate([X_train_disease] + new_examples)

from sklearn.preprocessing import OneHotEncoder
def onehot_encode(y, n_classes=None):
    yy = y.reshape(-1, 1)
    enc = OneHotEncoder(n_values=n_classes) if n_classes else OneHotEncoder()
    enc.fit(yy)
    one_hot_labels = enc.transform(yy).toarray()
    return one_hot_labels

y_train_disease = onehot_encode(np.array([disease_label_int for x in X_train_disease]), 15)

In [58]:
# train with the generated examples
print(y_train_sub.shape, y_train_disease.shape)
X_train, y_train = np.concatenate((X_train_sub, X_train_disease)), np.concatenate((y_train_sub, y_train_disease))

classifier = Classifier(trainable=True, n_classes=15)
classifier.fit(X_train, y_train)

(82089, 15) (20, 15)
Train on 73898 samples, validate on 8211 samples
Epoch 1/100
73898/73898 [==============================] - 5s 74us/step - loss: 2.4020 - acc: 0.1281 - val_loss: 4.9393 - val_acc: 0.1521
Epoch 2/100
73898/73898 [==============================] - 5s 69us/step - loss: 2.2429 - acc: 0.1596 - val_loss: 5.0925 - val_acc: 0.1510
Epoch 3/100
73898/73898 [==============================] - 5s 70us/step - loss: 2.1852 - acc: 0.1718 - val_loss: 5.2195 - val_acc: 0.2107
Epoch 4/100
73898/73898 [==============================] - 5s 70us/step - loss: 2.1422 - acc: 0.1784 - val_loss: 5.3376 - val_acc: 0.2301
Epoch 5/100
73898/73898 [==============================] - 5s 70us/step - loss: 2.1136 - acc: 0.1813 - val_loss: 5.4491 - val_acc: 0.1703
Epoch 6/100
73898/73898 [==============================] - 5s 71us/step - loss: 2.0852 - acc: 0.1848 - val_loss: 5.5945 - val_acc: 0.2124
Epoch 7/100
73898/73898 [==============================] - 5s 69us/step - loss: 2.0620 - acc: 0.1861 -

73898/73898 [==============================] - 5s 72us/step - loss: 1.6624 - acc: 0.2344 - val_loss: 7.8938 - val_acc: 0.2678
Epoch 60/100
73898/73898 [==============================] - 7s 92us/step - loss: 1.6585 - acc: 0.2369 - val_loss: 7.8704 - val_acc: 0.1925
Epoch 61/100
73898/73898 [==============================] - 5s 70us/step - loss: 1.6538 - acc: 0.2373 - val_loss: 7.8839 - val_acc: 0.1829
Epoch 62/100
73898/73898 [==============================] - 6s 75us/step - loss: 1.6489 - acc: 0.2357 - val_loss: 7.9032 - val_acc: 0.1767
Epoch 63/100
73898/73898 [==============================] - 5s 70us/step - loss: 1.6447 - acc: 0.2378 - val_loss: 7.9065 - val_acc: 0.1644
Epoch 64/100
73898/73898 [==============================] - 5s 70us/step - loss: 1.6400 - acc: 0.2373 - val_loss: 7.9341 - val_acc: 0.2117
Epoch 65/100
73898/73898 [==============================] - 5s 71us/step - loss: 1.6385 - acc: 0.2404 - val_loss: 7.9437 - val_acc: 0.1823
Epoch 66/100
73898/73898 [==============

In [59]:
loss, acc = classifier.evaluate(X_test_sub, y_test_sub)
print('accuracy for regular diseases is {0}'.format(acc))

loss, acc = classifier.evaluate(X_test_disease, y_test_disease)
print('accuracy for novel disease {0} with generated examples is {1}'.format(disease_label_int, acc))

9125/9125 [==============================] - 0s 37us/step
accuracy for regular diseases is 0.1907945205479452
109/109 [==============================] - 0s 55us/step
accuracy for novel disease 7 with generated examples is 0.0
